In [44]:
import csv
import boto3

# reading in the saved credential file
with open('rekog_demo_secret_credentials.csv', 'r') as input:
    next(input) #skip first row since it's just column names
    reader = csv.reader(input)
    for line in reader:
        access_key_id = line[2]
        secret_access_key = line[3]
        
choc_chip_cookie_img = 'choc_chip_cookie.jpeg' #locally saved basic photo
pink_sugar_cookie_img = 'pink_sugar_cookie.jpeg' #locally saved basic photo
face_upset_img = 'face_upset.png'
face_neutral_img = 'face_neutral.png'
face_happy_img = 'face_happy.png'

# using boto3 to use aws rekognition functionality using the 
# secret credentials saved in the csv and including region
client = boto3.client('rekognition', region_name='us-east-2',
                     aws_access_key_id = access_key_id,
                     aws_secret_access_key = secret_access_key)

# if you have a locally saved image, it needs to be converted
# to bytes and then that gets passed into the rekognition functionality
with open(choc_chip_cookie_img, 'rb') as choc_chip_cookie:
    choc_chip_cookie_bytes = choc_chip_cookie.read()
with open(pink_sugar_cookie_img, 'rb') as pink_sugar_cookie:
    pink_sugar_cookie_bytes = pink_sugar_cookie.read()

with open(face_upset_img, 'rb') as face_upset:
    face_upset_bytes = face_upset.read()
with open(face_neutral_img, 'rb') as face_neutral:
    face_neutral_bytes = face_neutral.read()
with open(face_happy_img, 'rb') as face_happy:
    face_happy_bytes = face_happy.read()
    
# perform rekognition task for ***LABELS*** and save label output to response
choc_chip_response = client.detect_labels(
    Image={'Bytes': choc_chip_cookie_bytes},
    MaxLabels=10)
pink_sugar_response = client.detect_labels(
    Image={'Bytes': pink_sugar_cookie_bytes},
    MaxLabels=10)

# perform rekognition task for ***FACIAL ATTRIBUTES*** and save label output to response
# by default, the rekognition task will only give info on bounding box etc
# To get useful information such as emotions, age range, etc you need to switch Attributes to ALL
face_upset_response = client.detect_faces(
    Image={'Bytes': face_upset_bytes},
    Attributes = ['ALL'])
face_neutral_response = client.detect_faces(
    Image={'Bytes': face_neutral_bytes},
    Attributes = ['ALL'])
face_happy_response = client.detect_faces(
    Image={'Bytes': face_happy_bytes},
    Attributes = ['ALL'])


In [45]:
print(choc_chip_response)


{'Labels': [{'Name': 'Bread', 'Confidence': 96.03101348876953, 'Instances': [{'BoundingBox': {'Width': 0.304768443107605, 'Height': 0.30535101890563965, 'Left': 0.34730684757232666, 'Top': 0.11515714228153229}, 'Confidence': 96.03101348876953}, {'BoundingBox': {'Width': 0.26449480652809143, 'Height': 0.2986818552017212, 'Left': 0.7341976761817932, 'Top': 0.15861476957798004}, 'Confidence': 92.96229553222656}, {'BoundingBox': {'Width': 0.24845807254314423, 'Height': 0.1247088834643364, 'Left': 0.7449105381965637, 'Top': 0.0}, 'Confidence': 92.55546569824219}, {'BoundingBox': {'Width': 0.2908284366130829, 'Height': 0.09654789417982101, 'Left': 0.38053974509239197, 'Top': 0.0}, 'Confidence': 81.6482925415039}, {'BoundingBox': {'Width': 0.2957373559474945, 'Height': 0.1747705191373825, 'Left': 0.004738384857773781, 'Top': 0.0}, 'Confidence': 81.46404266357422}, {'BoundingBox': {'Width': 0.27996426820755005, 'Height': 0.2973284125328064, 'Left': 0.2769024670124054, 'Top': 0.4653069078922272

In [46]:
print('Detected labels:' )
for label in choc_chip_response['Labels']:
    print(label['Name'] + '\t' + str(round(label['Confidence'], 4)))

Detected labels:
Bread	96.031
Food	96.031
Cookie	96.0159
Plant	82.2473
Bakery	72.3106
Shop	72.3106
Sweets	71.338
Dessert	67.9503
Cracker	67.4931
Vegetable	55.2718


In [47]:
print('Detected labels:' )
for label in pink_sugar_response['Labels']:
    print(label['Name'] + '\t' + str(round(label['Confidence'], 4)))

Detected labels:
Burger	96.0284
Food	96.0284
Sweets	95.5612
Bakery	94.7587
Dessert	93.2565
Cream	91.856
Sprinkles	89.3898
Cake	84.3875
Icing	79.6774
Pastry	74.5613


In [48]:
print(face_upset_response)

{'FaceDetails': [{'BoundingBox': {'Width': 1.1023614406585693, 'Height': 0.8992219567298889, 'Left': 0.013540845364332199, 'Top': 0.07032295316457748}, 'AgeRange': {'Low': 23, 'High': 35}, 'Smile': {'Value': True, 'Confidence': 84.34361267089844}, 'Eyeglasses': {'Value': False, 'Confidence': 99.33621215820312}, 'Sunglasses': {'Value': False, 'Confidence': 99.8534927368164}, 'Gender': {'Value': 'Female', 'Confidence': 87.06004333496094}, 'Beard': {'Value': False, 'Confidence': 92.0462646484375}, 'Mustache': {'Value': False, 'Confidence': 97.94818115234375}, 'EyesOpen': {'Value': False, 'Confidence': 92.97705078125}, 'MouthOpen': {'Value': True, 'Confidence': 74.54232788085938}, 'Emotions': [{'Type': 'SAD', 'Confidence': 60.39643096923828}, {'Type': 'FEAR', 'Confidence': 19.238985061645508}, {'Type': 'HAPPY', 'Confidence': 13.093246459960938}, {'Type': 'CONFUSED', 'Confidence': 1.9378507137298584}, {'Type': 'SURPRISED', 'Confidence': 1.6602569818496704}, {'Type': 'ANGRY', 'Confidence': 1

In [49]:
print('Detected labels:' )
for label in face_upset_response['FaceDetails']:
    print('Age Range: ')
    print(label['AgeRange'] )
    print('Smile: ' )
    print(str(label['Smile']['Value']) + '\t' + str(round(label['Smile']['Confidence'], 4)))
    print('Emotions: ' )
    for emo in label['Emotions']:
        print(emo['Type'] + '\t' + str(round(emo['Confidence'], 4)))

Detected labels:
Age Range: 
{'Low': 23, 'High': 35}
Smile: 
True	84.3436
Emotions: 
SAD	60.3964
FEAR	19.239
HAPPY	13.0932
CONFUSED	1.9379
SURPRISED	1.6603
ANGRY	1.4859
DISGUSTED	1.3493
CALM	0.838


In [50]:
print('Detected labels:' )
for label in face_happy_response['FaceDetails']:
    print('Age Range: ')
    print(label['AgeRange'] )
    print('Smile: ' )
    print(str(label['Smile']['Value']) + '\t' + str(round(label['Smile']['Confidence'], 4)))
    print('Emotions: ' )
    for emo in label['Emotions']:
        print(emo['Type'] + '\t' + str(round(emo['Confidence'], 4)))

Detected labels:
Age Range: 
{'Low': 22, 'High': 34}
Smile: 
True	98.0314
Emotions: 
HAPPY	92.5233
SURPRISED	3.0949
CALM	1.3765
CONFUSED	1.1927
DISGUSTED	0.6064
SAD	0.4318
ANGRY	0.3957
FEAR	0.3787


In [51]:
print('Detected labels:' )
for label in face_neutral_response['FaceDetails']:
    print('Age Range: ')
    print(label['AgeRange'] )
    print('Smile: ' )
    print(str(label['Smile']['Value']) + '\t' + str(round(label['Smile']['Confidence'], 4)))
    print('Emotions: ' )
    for emo in label['Emotions']:
        print(emo['Type'] + '\t' + str(round(emo['Confidence'], 4)))

Detected labels:
Age Range: 
{'Low': 22, 'High': 34}
Smile: 
False	99.7166
Emotions: 
CALM	89.9823
SAD	9.5926
ANGRY	0.1623
CONFUSED	0.1329
SURPRISED	0.0772
HAPPY	0.0243
DISGUSTED	0.0159
FEAR	0.0126
